In [1]:
def calculateIndicators (ticker,):

    #from merged prices import data considering  on the ticker 
    #set query
    query = f"""
                SELECT *
                FROM `stockmarket-v0.stockMarket_dev.mergedPrices` as df
                where df.symbol = '{ticker}'
                order by df.date

                """

    #import data 
    df_returns = pd.read_gbq(query= query,
                            project_id=project_id, ).set_index(['symbol', 'date',])
    
    nr_days=  21
    month_list = [1,2,3,4,5,6]
    df_returns['dailyReturns'] = df_returns['close'].pct_change()
    df_returns['dailyIndexReturns'] = df_returns['indexPrice'].pct_change()
    corr = (df_returns[['dailyReturns','dailyIndexReturns']]).corr().iloc[0,1]

    for month_ in month_list:
        n = nr_days * month_
        df_returns[f'MA_{n}'] = df_returns['dailyReturns'].rolling(n).mean()
        df_returns[f'Volatility_{n}'] = df_returns['dailyReturns'].rolling(n).std()
        
        df_returns[f'excessReturn_{n}'] = df_returns[f'MA_{n}'] - df_returns['dailyIndexReturns'].rolling(n).mean()
        
        df_returns[f'beta_{n}'] = corr * (df_returns[f'Volatility_{n}'] /df_returns['dailyIndexReturns'].rolling(n).std() )
        
        
    cols = [ 'MA_21',
        'Volatility_21', 'excessReturn_21', 'beta_21', 'MA_42', 'Volatility_42',
        'excessReturn_42', 'beta_42', 'MA_63', 'Volatility_63',
        'excessReturn_63', 'beta_63', 'MA_84', 'Volatility_84',
        'excessReturn_84', 'beta_84', 'MA_105', 'Volatility_105',
        'excessReturn_105', 'beta_105', 'MA_126', 'Volatility_126',
        'excessReturn_126', 'beta_126'
        ]
    df_returns = df_returns[cols]
    df_returns = df_returns.iloc[n:]
    
    return(df_returns.reset_index())


import sys
sys.path.insert(1, r'C:\Users\rafae\Personal\Github\GCP-Stock_Market\FMP_API')
sys.path.insert(2, r'C:\Users\rafae\Personal\Github\GCP-Stock_Market\InsertData')


import pandas as pd


import FMP_API as API
import insertBQ_Class as insert

#set key
key = '54594d7278e0fa3c0831a72c60e04b8d'

#set big query variables
project_id = 'stockmarket-v0'
table_name = 'TechnicalIndicator'
dataset_name = 'stockMarket_dev'

#set table path
destionation_table_name = f'{project_id}.{dataset_name}.{table_name}'

#get ticker object
tickerAPI = API.getSymbols(key)

#get tickers
tickers = tickerAPI.get_NYSE()

#print general information
print(f'Project ID: {project_id}')
print(f'Dataset Name: {dataset_name}')
print(f'Inserting Data into: {table_name}')

#create an empty df
data = pd.DataFrame()
#for each ticker 
for ticker in tickers[:1]:  
    try:
        #calculate alpha for n = 63 (quarter)
        data_temp  = calculateIndicators(ticker,)
        #add to the main dataframe
        data = pd.concat([data,data_temp ],
                        axis=0)
    except Exception as error:
        print(f'Ticker: {ticker}')
        print(error)
        pass
    
#set numberical columns to float
cols = ['MA_21',
 'Volatility_21',
 'excessReturn_21',
 'beta_21',
 'MA_42',
 'Volatility_42',
 'excessReturn_42',
 'beta_42',
 'MA_63',
 'Volatility_63',
 'excessReturn_63',
 'beta_63',
 'MA_84',
 'Volatility_84',
 'excessReturn_84',
 'beta_84',
 'MA_105',
 'Volatility_105',
 'excessReturn_105',
 'beta_105',
 'MA_126',
 'Volatility_126',
 'excessReturn_126',
 'beta_126']
data[cols] = data[cols].astype(float)

#set query to check for duplicates
query =  f"""
    SELECT df.symbol,df.date,df.MA_21
    FROM `{destionation_table_name}` as df 
    
    """

#set index for duplicate checking
index_list = ['symbol','date']

# set insert data object
insertBQ = insert.insertData(projectID = project_id,
                            dataset_name = dataset_name,
                            duplicates = True,)

#insert data
insertBQ.insertData(data = data,
                    table_name = table_name,
                    query = query,
                    index_list = index_list)

Project ID: stockmarket-v0
Dataset Name: stockMarket_dev
Inserting Data into: TechnicalIndicator
